In [36]:
#!/usr/bin/env python
"""
@author: Jayant Gupta
last update: 02.06.2021
"""

'\n@author: Jayant Gupta\nlast update: 02.06.2021\n'

In [37]:
# Import necessary libraries
import tensorflow as tf
import os
import random
import numpy as np

import models
 
from tqdm import tqdm 
import json

from skimage.io import imread, imshow, show
from skimage.transform import resize
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from PIL import Image

# Handles reading large number of files.
Image.MAX_IMAGE_PIXELS = 219494175 

In [38]:
seed = 42
np.random.seed = seed

IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 4
IMG_INDEX = "ExG"

# This function creates a dictionary to store images and their mask.
# This function is curated to handle train images *.tif format and 
# mask file in *.jpeg format
def get_in_out_dict(Image_Path, Mask_Path, Index_Path=None, Index_Type=None):
    img_fns = os.listdir(Image_Path)
    mask_fns = os.listdir(Mask_Path)

    if IMG_CHANNELS > 3:
        indices_fns = os.listdir(Index_Path)

    in_out_dict = {}
    for img_filename in img_fns:
        key = img_filename.split('.')[0]
        in_out_dict[key] = []
        in_out_dict[key].append(
            os.path.join(Image_Path, img_filename))
        if IMG_CHANNELS > 3:
            in_out_dict[key].append(
                os.path.join(Index_Path, [index for index in indices_fns if key in index and Index_Type in index][0]))
        in_out_dict[key].append(
            os.path.join(Mask_Path, [mask for mask in mask_fns if key in mask][0]))
    
    for key in list(in_out_dict.keys())[0:5]:
        print(in_out_dict[key])

    return in_out_dict

# #train_mask_pair = {**train_mask_mpls_pair, **train_mask_stp_pair} # For OSFA
# #test_mask_pair = {**test_mask_mpls_pair, **test_mask_stp_pair}    # For OSFA 
Mask_Path = json.load(open('config.json'))['filepaths']['default_mask_folder']
Image_Path = json.load(open('config.json'))['filepaths']['input_imagery_dir']
in_out_dict = {}
if IMG_CHANNELS > 3:
    Index_Path = json.load(open('config.json'))['filepaths']['input_vi_dir']
    in_out_dict = get_in_out_dict(Image_Path, Mask_Path, Index_Path, IMG_INDEX)
else:
    in_out_dict = get_in_out_dict(Image_Path, Mask_Path)

['datasets/MN_raster_Hennepin_North\\120_23_13_01.tif', 'datasets/MN_Hennepin_North_vi\\120_23_13_01_ExG.tif', 'outputs/MN_raster_Hennepin_North_Mask\\120_23_13_01_mask.jpeg']
['datasets/MN_raster_Hennepin_North\\120_23_13_02.tif', 'datasets/MN_Hennepin_North_vi\\120_23_13_02_ExG.tif', 'outputs/MN_raster_Hennepin_North_Mask\\120_23_13_02_mask.jpeg']
['datasets/MN_raster_Hennepin_North\\120_23_13_03.tif', 'datasets/MN_Hennepin_North_vi\\120_23_13_03_ExG.tif', 'outputs/MN_raster_Hennepin_North_Mask\\120_23_13_03_mask.jpeg']
['datasets/MN_raster_Hennepin_North\\120_23_13_04.tif', 'datasets/MN_Hennepin_North_vi\\120_23_13_04_ExG.tif', 'outputs/MN_raster_Hennepin_North_Mask\\120_23_13_04_mask.jpeg']
['datasets/MN_raster_Hennepin_North\\120_23_14_01.tif', 'datasets/MN_Hennepin_North_vi\\120_23_14_01_ExG.tif', 'outputs/MN_raster_Hennepin_North_Mask\\120_23_14_01_mask.jpeg']


In [39]:
def join_image_and_index(img_path, index_path):
    image = Image.open(img_path) # Using Pillow Image object
    image = image.resize((IMG_HEIGHT, IMG_WIDTH)) # 
    image_array = np.asarray(image)/255.

    index = Image.open(index_path) # Using Pillow Image object
    index = index.resize((IMG_HEIGHT, IMG_WIDTH))
    index_array = np.asarray(index)
    return np.dstack((image_array, index_array))

# Function to read the data from image_mask_pair file paths.
def read_data(in_out_dict):
    X_train = np.zeros((len(in_out_dict), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.float32)
    Y_train = np.zeros((len(in_out_dict), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

    keys = in_out_dict.keys()

    print('Resizing training images and masks')
    n = 0
    for key, value in tqdm(in_out_dict.items()):
        print(key)
        if IMG_CHANNELS > 3:
            [img_path, index_path, mask_path] = value
            X_train[n] = join_image_and_index(img_path, index_path)
        else:
            [img_path, mask_path] = value
            image = Image.open(img_path) # Using Pillow Image object
            image = image.resize((IMG_HEIGHT, IMG_WIDTH)) # 
            X_train[n] = np.asarray(image)/255.
        mask = Image.open(mask_path) # Using Pillow Image object
        mask = mask.resize((IMG_HEIGHT, IMG_WIDTH))
        mask = np.array(mask)
        mask = mask.reshape(256, 256, 1)
        mask_bool = mask > 0          
        Y_train[n] = mask_bool 
        n += 1
    return X_train, Y_train

X_train_in, Y_train_in = read_data(in_out_dict)

  0%|          | 0/36 [00:00<?, ?it/s]Resizing training images and masks
120_23_13_01
100%|██████████| 36/36 [03:19<00:00,  5.53s/it]


In [40]:
# Need to run once. Generate training, validation, and test dataset.
def generate_train_val_test(X_train, Y_train):
    X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size = 0.1, random_state = 1)
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.12, random_state = 1)
    print("Train Size = {0}, Test size = {1}, Validation size = {2}".format(len(X_train), len(X_test), len(X_val)))
    return X_train, Y_train, X_val, Y_val, X_test, Y_test

X_train_split, Y_train_split, X_val, Y_val, X_test, Y_test = generate_train_val_test(X_train_in, Y_train_in)

outfile = json.load(open('config.json'))['filepaths']['default_pyobjects'] + '/Hennepin_North.npz'
np.savez(outfile ,X_train=X_train_split, Y_train=Y_train_split, X_val=X_val, Y_val=Y_val, X_test=X_test, Y_test=Y_test)
npzfile = np.load(outfile)
print(npzfile.files)
X_train = npzfile['X_train']
Y_train = npzfile['Y_train']
X_test = npzfile['X_test']
Y_test = npzfile['Y_test']
X_val = npzfile['X_val']
Y_val = npzfile['Y_val']

Train Size = 28, Test size = 4, Validation size = 4
['X_train', 'Y_train', 'X_val', 'Y_val', 'X_test', 'Y_test']


In [ ]:
# Samples the input data.
for (X, Y, Title) in [(X_train, Y_train, "Train"), (X_test, Y_test, "Test"), (X_val, Y_val, "Validation")]:
    print("Sampling the " + Title + " data")
    image_x = random.randint(0, len(X)-1)
    plt.imshow(X[image_x][:,:,:3])
    plt.title("RGB for "+Title)
    plt.figure()
    for c in range(IMG_CHANNELS-3):
        art = plt.imshow(X[image_x][:,:,3:4+c], cmap ='plasma')
        plt.colorbar(art)
        plt.title("VI for "+Title+" ("+IMG_INDEX+")")
        plt.figure()
    plt.imshow(np.squeeze(Y[image_x]))
    plt.title("Mask label for "+Title)
    plt.show()

In [ ]:
#class CustomCallback(keras.callbacks.Callback):
#    def on_train_batch_begin(self, batch, logs=None):
#        if not hasattr(self.model.optimizer, "lr"):
#            raise ValueError('Optimizer must have a "lr" attribute.')
#        lr = float(tf.keras.backend.get_value(self.model.optimizer_learning_rate))
# sample_weights        

# Generate a UNetS model
unets_model = models.UNetS(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='logs'),
        tf.keras.callbacks.ModelCheckpoint(filepath=json.load(open('config.json'))['filepaths']['default_checkpoints'] + '/MPLS-UNetS.h5', verbose=1, save_best_only=True)]

# Check sample weighting
sample_weight = np.ones(shape=(len(Y_train),))

results = unets_model.fit(X_train, Y_train, sample_weight = sample_weight, validation_data=(X_val, Y_val), batch_size=8, epochs=10, callbacks=callbacks)

####################################

In [ ]:
# Generate a UNet model
unet_model = models.UNet(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

################################

#class CustomCallback(keras.callbacks.Callback):
#    def on_train_batch_begin(self, batch, logs=None):
#        if not hasattr(self.model.optimizer, "lr"):
#            raise ValueError('Optimizer must have a "lr" attribute.')
#        lr = float(tf.keras.backend.get_value(self.model.optimizer_learning_rate))
# sample_weights        

#Modelcheckpoint
#checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='/Users/jayantgupta/SVANN/model_for_nuclei.h5', verbose=1, save_best_only=True)

callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='logs'),
        tf.keras.callbacks.ModelCheckpoint(filepath=json.load(open('config.json'))['filepaths']['default_checkpoints'] + '/MPLS-UNet.h5', verbose=1, save_best_only=True)]

# Check sample weighting
#sample_weight = np.ones(shape=(len(Y_train),))

results = unet_model.fit(X_train, Y_train, sample_weight = sample_weight, validation_split=0.1, batch_size=16, epochs=3, callbacks=callbacks)

####################################

In [62]:
unets_model = models.UNetS(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
weight_file = json.load(open('config.json'))['filepaths']['default_checkpoints']+'/'+IMG_INDEX+'/MPLS-UNetS.h5'
unets_model.load_weights(weight_file)

THRESHOLD_TYPE = ['match', 'mean', 'median', 'static']
preds_train = unets_model.predict(X_train, verbose=1)
preds_val = unets_model.predict(X_val, verbose=1)
preds_test = unets_model.predict(X_test, verbose=1)

for threshold_type in THRESHOLD_TYPE:
    threshold_train=np.percentile(preds_train, (1-np.mean(Y_train.reshape(Y_train.shape[0], Y_train.shape[1]*Y_train.shape[2]), axis=1))*100)
    threshold_val=np.percentile(preds_val, (1-np.mean(Y_val.reshape(Y_val.shape[0], Y_val.shape[1]*Y_val.shape[2]), axis=1))*100)
    threshold_test=np.percentile(preds_test, (1-np.mean(Y_test.reshape(Y_test.shape[0], Y_test.shape[1]*Y_test.shape[2]), axis=1))*100)

    threshold_mean = np.mean(threshold_train)
    threshold_median = np.median(threshold_train)

    if threshold_type=='mean':
        threshold_train = np.full(len(preds_train), threshold_mean)
        threshold_val = np.full(len(preds_val), threshold_mean)
        threshold_test = np.full(len(preds_test), threshold_mean)
    elif threshold_type=='median':
        threshold_train = np.full(len(preds_train), threshold_median)
        threshold_val = np.full(len(preds_val), threshold_median)
        threshold_test = np.full(len(preds_test), threshold_median)
    elif threshold_type=='static':
        threshold_train = np.full(len(preds_train), 0.2)
        threshold_val = np.full(len(preds_val), 0.2)
        threshold_test = np.full(len(preds_test), 0.2)

    preds_train_t = (np.squeeze(preds_train) > threshold_train[:, None, None]).astype(np.uint8)
    preds_val_t = (np.squeeze(preds_val) > threshold_val[:, None, None]).astype(np.uint8)
    preds_test_t = (np.squeeze(preds_test) > threshold_test[:, None, None]).astype(np.uint8)

    # Perform a sanity check on some random training samples
    for ix in range(len(preds_train_t)):
        plt.ioff()
        fig, axs = plt.subplots(2, 3, figsize=(16, 10), squeeze=True)
        fig.suptitle('%s with %s threshold: %0.3f' % (IMG_INDEX, threshold_type, threshold_train[ix]))
        axs[0, 0].imshow(X_train[ix][:,:,:3])
        axs[0, 0].set_title("Input imagery")
        axs[0, 1].imshow(X_train[ix][:,:,3:4])
        axs[0, 1].set_title("Input index")
        axs[0, 2].imshow(Y_train[ix])
        axs[0, 2].set_title("Binary mask label")
        axs[1, 0].imshow(preds_train[ix])
        axs[1, 0].set_title('Prediction values')
        axs[1, 1].imshow(preds_train_t[ix])
        axs[1, 1].set_title('Binary prediction')
        cm = np.squeeze(Y_train[ix]).astype(int)-2*np.squeeze(preds_train_t[ix]).astype(int)
        tn_, fp_, fn_, tp_ = confusion_matrix(Y_train[ix].astype(int).flatten(), preds_train_t[ix].astype(int).flatten(), labels=[0,1]).ravel()
        precision = tp_/(tp_+fp_)
        recall = tp_/(tp_+fn_)
        f1 = 2*(precision*recall)/(precision+recall)
        axs[1, 2].imshow(cm, cmap='RdBu', interpolation='gaussian')
        axs[1, 2].set_title('Confusion Matrix (f1: %0.3f)\nLight Red=tp, Light Blue=tn, Red=fp, Blue=fn' % (f1))
        plt.savefig('outputs/plots/'+str(ix)+'/UNetS_'+IMG_INDEX+'_'+threshold_type+'_'+str(ix))
        break


1/1 [==============================] - 0s 179ms/step


In [ ]:
# Perform a sanity check on some random validation samples
ix = random.randint(0, len(X_val)-1)
imshow(X_val[ix][:,:,:3])
plt.title("Input imagery")
plt.show()
imshow(np.squeeze(Y_val[ix]))
plt.title("Binary mask label")
plt.show()
imshow(np.squeeze(preds_val_t[ix]))
plt.title('Binary prediction')
plt.show()

In [ ]:
tn = 0.
fp = 0.
fn = 0.
tp = 0.
for ix in range(0, len(preds_test_t)):
    tn_, fp_, fn_, tp_ = confusion_matrix(Y_test[ix].astype(int).flatten(), preds_test_t[ix].astype(int).flatten(), labels=[0,1]).ravel()
    tn = tn + tn_
    fp = fp + fp_
    fn = fn + fn_
    tp = tp + tp_

print(tn, fp, fn, tp)

In [ ]:
tn = 0.
fp = 0.
fn = 0.
tp = 0.
for ix in range(0, len(preds_val_t)):
    #print(ix)
    tn_, fp_, fn_, tp_ = confusion_matrix(Y_val[ix].astype(int).flatten(), preds_val_t[ix].astype(int).flatten(), labels=[0,1]).ravel()
    tn = tn + tn_
    fp = fp + fp_
    fn = fn + fn_
    tp = tp + tp_
#    print(tn, fp, fn, tp)

print(tn, fp, fn, tp)

In [ ]:
# Selecting test data samples
# Need to run once at the start
# print(img_mask_HCL_pair.keys)

# import shutil

# TEST_HCL_PATH = '/Users/jayantgupta/Desktop/Imagery/Wetland_Imagery/027_23_05_020/Test/img'
# TEST_HCL_MASK_PATH = '/Users/jayantgupta/Desktop/Imagery/Wetland_Imagery/027_23_05_020/Test/mask'

# TEST_HCR_PATH = '/Users/jayantgupta/Desktop/Imagery/Wetland_Imagery/027_23_05_021/Test/img'
# TEST_HCR_MASK_PATH = '/Users/jayantgupta/Desktop/Imagery/Wetland_Imagery/027_23_05_021/Test/mask'

# test_samples_HCL = random.choices(list(img_mask_HCL_pair.keys()), k=98)
# for sample in test_samples_HCL:
#     try:
#         img_path = img_mask_HCL_pair[sample][0]
#         img_test_path = os.path.join(TEST_HCL_PATH,img_path.split('/')[-1])
#         shutil.move(img_path, img_test_path)
#     #print(img_path)
#     #print(img_test_path)

#         mask_path = img_mask_HCL_pair[sample][1]
#         mask_test_path = os.path.join(TEST_HCL_MASK_PATH, mask_path.split('/')[-1])
#         shutil.move(mask_path, mask_test_path)
#     except:
#         print(sample)
#     #break;

# test_samples_HCR = random.choices(list(img_mask_HCR_pair.keys()), k=98)
# for sample in test_samples_HCR:
#     try:
#         img_path = img_mask_HCR_pair[sample][0]
#         img_test_path = os.path.join(TEST_HCR_PATH,img_path.split('/')[-1])
#         shutil.move(img_path, img_test_path)
    
#         mask_path = img_mask_HCR_pair[sample][1]
#         mask_test_path = os.path.join(TEST_HCR_MASK_PATH, mask_path.split('/')[-1])
#         shutil.move(mask_path, mask_test_path)    
#     except:
#         print(sample)